# WIP Development of Pre-processing Steps (003)
 - 20190315
 - MJP
 - Work towards processing many years worth of data 

In [1]:
%load_ext autoreload
%matplotlib inline

# Standard imports 
import numpy as np 
import matplotlib.pyplot as plt 
import healpy as hp
import sys, os
from pathlib import Path


import MPC_library as MPCL
import phys_const as PHYS
import mpchecker 
import classes as Classes
import params 
import angles as ANG 
import IO 
import ele220 

/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matthewjohnpayne/anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Some of the results returned below might depend on the assumed HP scale for the preprocessing
 - E.g. the HP # for the center of the pointing ...
 - At the time of evaluation the assumed parameters are ...

In [2]:
print("Healpix: Nside", params.hp_nside)
print("Healpix: Npix", params.hp_npix)
print("Healpix: Area [deg^2]", params.hp_area)
print("Healpix: Side [deg]", params.hp_side)

Healpix: Nside 256
Healpix: Npix 786432
Healpix: Area [deg^2] 0.05245585282569793
Healpix: Side [deg] 0.22903242745449373


# Processing Many Nights / Epochs 
 - Concentrate on the numbered and/or MBA objects for now
 - Want to iterate through each 40-day epoch
 - Want to produce *STATES* every 40 days
 - Want to produce *HP* every night
 - Want to understand how accurate / inaccurate the propagation of the states is (and whether there's any need to interpolate / average the elements)

In [5]:
# Get array of available epochs ...
# ... just looking at 'mpn' files for now
nbody_data_dir = 'nbody_data/elements'
nbodyFileDict = { f:None for f in os.listdir(nbody_data_dir) if f[:3]=='mpn'}

# Convert to JD 
garethEpochs = sorted(set([k[k.find("K"):k.find("K")+5] for k in nbodyFileDict.keys() ]))[-22:]
jds          = np.array([MPCL.yrmndy2JD(MPCL.convertEpoch(garethEpoch)) for garethEpoch in garethEpochs ])

# Get range of "eligibility" for each epoch
# - I am deliberately being conservative about the end-points 
start = [(jds[i-1]+jd)/2. if i !=0         else jd for i, jd in enumerate(jds) ]
end   = [(jds[i+1]+jd)/2. if i <len(jds)-1 else jd for i, jd in enumerate(jds) ]

# Hack to get and load TESS data
TESS_DATA = IO.parse_TESS_HORIZONS('TESS_HORIZONS.txt')
TESS_DICT = { str(round(v[0],1)).replace(".","_"): v[1:4] for v in TESS_DATA}
 
# Loop over nbody files ...
for i, garethEpoch in enumerate(garethEpochs ) :
    print(garethEpoch) 
    
    # Read the data 
    nbodyFileDict = IO.get_nbody_data(garethEpoch)
    print("nbodyFileDict.keys", list(nbodyFileDict.keys()) )
    
    # Convert to states at epoch of nbody file
    orbitObjectList = []
    for k,data in nbodyFileDict.items():
        for line in data:
            # We'll save the data into an "OrbitObject"
            OO = Classes.OrbitObject()

            # Parse line using ele220
            # Get cart_state from ele220 keplerian elements
            OO.generate_cart_state_via_ele220(MPCL.Constants.GMsun, ele220.Ele220(line))

            # Save into OrbitObject into list
            orbitObjectList.append(OO)

    print("Read %d lines from nbody files..."%len(orbitObjectList))
    
    # Save OrbitObject data (at epoch of nbody file)
    IO.save_OrbitObjects(orbitObjectList,  str(round(jds[i],1)).replace(".","_") )

   # Propagate to range of nights "allowed" for each nbody epoch
    # Calculate & save HP-arrays for each night (and for each center where relevant)
    # (i) Define the range of nights ...
    JDUTC_    = np.arange(start[i], end[i], 1)
    JDUTCstr_ = [str(round(JD,1)).replace(".","_")  for JD in JDUTC_]

    # (ii) Loop over obs-code centers 
    for oc in ['-95' , '500']:
        
        # (A) Get the observatory positions (ECLIPTIC w.r.t. Sun)
        # - We need to do this for all of the nights we are propagating to ...
        if oc in ['-95']: # If TESS, handle separately (for now)
            posEC_OBS_wrt_HELIO = np.array( [TESS_DICT[JDstr] for JDstr in JDUTCstr_ if JDstr in TESS_DICT])
        else:
            posEQ_OBS_wrt_HELIO = np.array( [ MPCL.Observatory().getObservatoryPosition(oc, JDUTC) for JDUTC in JDUTC_ ])
            posEC_OBS_wrt_HELIO = np.array( [np.dot(np.transpose(PHYS.rot_mat), item ) for item in posEQ_OBS_wrt_HELIO ])

        if posEC_OBS_wrt_HELIO != []:
            # (B) Loop over OrbitObjects & get HP/UV/RADEC at each time (night)
            listOrbitNumberByHP = [[[] for i in range(params.hp_npix)] for j in range(len(JDUTC_))]
            for i, OO in enumerate(orbitObjectList):            
                # Get healpix (list: one for each time)
                # Invert to store OrbitObjects by HP
                # - Doing the loop this way might be a little slow ...
                for j, HP in enumerate(OO.get_healpix( JDUTC_ , posEC_OBS_wrt_HELIO )):
                    listOrbitNumberByHP[ j ][HP].append(i)

            # Save file/database
            IO.save_OrbitNumberByHPByDay(listOrbitNumberByHP, JDUTCstr_, oc )


K177Q
nbodyFileDict.keys ['mpu_K177Q.txt', 'mpn_add_K177Q.txt', 'mpn_K177Q.txt', 'mpu_add_K177Q.txt']
Read 716829 lines from nbody files...
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/orbitObject_states_2457960_5.npy'
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/orbitObject_desigs_2457960_5.npy'


/Users/matthewjohnpayne/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:63: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
/Users/matthewjohnpayne/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:63: DeprecationWarning: elementwise != comparison failed; this will raise an error in the future.


HERE save_OrbitNumberByHPByDay
2457960_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457960_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457960_5.npy'
2457961_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457961_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457961_5.npy'
2457962_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457962_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457962_5.npy'
2457963_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457963_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457963_5.npy'
2457964_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457964_5.npy
Sa

2457997_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457997_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457997_5.npy'
2457998_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457998_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457998_5.npy'
2457999_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457999_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2457999_5.npy'
2458000_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458000_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458000_5.npy'
2458001_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458001_5.npy
Saving '/Users/matthewjohnpayne/D

2458034_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458034_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458034_5.npy'
2458035_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458035_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458035_5.npy'
2458036_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458036_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458036_5.npy'
2458037_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458037_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458037_5.npy'
2458038_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458038_5.npy
Saving '/Users/matthewjohnpayne/D

...

2458731_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458731_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458731_5.npy'
2458732_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458732_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458732_5.npy'
2458733_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458733_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458733_5.npy'
2458734_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458734_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458734_5.npy'
2458735_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458735_5.npy
Saving '/Users/matthewjohnpayne/D

2458730_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458730_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458730_5.npy'
2458731_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458731_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458731_5.npy'
2458732_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458732_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458732_5.npy'
2458733_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458733_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458733_5.npy'
2458734_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458734_5.npy
Saving '/Users/matthewjohnpayne/D

2458767_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458767_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458767_5.npy'
2458768_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458768_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458768_5.npy'
2458769_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458769_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458769_5.npy'
2458770_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458770_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458770_5.npy'
2458771_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_-95_2458771_5.npy
Saving '/Users/matthewjohnpayne/D

2458766_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458766_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458766_5.npy'
2458767_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458767_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458767_5.npy'
2458768_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458768_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458768_5.npy'
2458769_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458769_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458769_5.npy'
2458770_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458770_5.npy
Saving '/Users/matthewjohnpayne/D

2458783_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458783_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458783_5.npy'
2458784_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458784_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458784_5.npy'
2458785_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458785_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458785_5.npy'
2458786_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458786_5.npy
Saving '/Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458786_5.npy'
2458787_5 786432 /Users/matthewjohnpayne/Dropbox/mpchecker/preprocessed_data/OrbitNumberByHP_500_2458787_5.npy
Saving '/Users/matthewjohnpayne/D